In [10]:
import requests
from bs4 import BeautifulSoup
import re
import os
from time import sleep
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime

In [11]:
import sqlite3
from sqlite3 import Error
import pandas as pd
#Name of Database File that Will be Created
database = "transcripts1.db"

In [12]:
#Create SQL Lite Connection
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn

In [13]:
#Connect and Create Table Transcripts if the Table Doesn't Exist
conn = create_connection(database)
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS TRANSCRIPTS ([TRANSCRIPTS] text PRIMARY KEY, [TICKER] text, [DATE] date,[TITLE] text)''')    
conn.commit()

In [14]:
#Create list of tickers grouped up by exhange
nsdq_ticks = "data/nasdaqlisted.txt"
nyse_ticks = "data/companylist.csv"
nyse = pd.read_csv(nyse_ticks)
nsdq = pd.read_csv(nsdq_ticks,sep="|")
nsdq = nsdq[nsdq['Security Name'].str.contains("Common Stock",na=False)]
nsdq = nsdq[nsdq['ETF'].str.contains("N",na=False)]
nyse_tickers = list(nyse['Symbol']) 
nsdq_tickers = list(nsdq['Symbol'])
nsdq_tickers[192]

'ATOS'

In [26]:
#Search through tickers in nasdaq list, load firefox browser and downloaded earning transcript text  
transcripts = []
ticks = []

for i in nsdq_tickers[194:]:
    try:
        for j in range(1,8):  
            link ='https://www.fool.com/quote/nasdaq/apple/{}/earnings-call-transcripts'.format(i)
            print(link)
            driver = webdriver.Firefox()
            driver.get(link)
            sleep(3)
            element = driver.find_element_by_xpath("/html/body/div[1]/div[2]/div/div/div[3]/div[2]/div[6]/div/div/article[{}]/div[2]/h4/a".format(j))
            element.click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            if "Earnings" in soup.find('h1').getText():  
                title = soup.find('h1').getText()
                transcript_soup = soup.find_all('p')[3:]            
                try:
                    date = soup.find("span", id="date").getText()
                except:
                    date = soup.find('h2').getText()
                transcript = [i.getText() for i in transcript_soup]
                c.execute('INSERT INTO TRANSCRIPTS VALUES (?,?,?,?)',[str(transcript),str(i),str(date),str(title)])
                conn.commit()
                driver.close()
            else:
                pass
    except:
        driver.close()
        pass

In [32]:
#Search through tickers in nyse list, load firefox browser and downloaded earning transcript text 
for i in nyse_tickers[1501:]:
    try:
        for j in range(1,8):    
            link ='https://www.fool.com/quote/nyse/apple/{}/earnings-call-transcripts'.format(i)
            driver = webdriver.Firefox()
            driver.get(link)
            sleep(2)
            element = driver.find_element_by_xpath("/html/body/div[1]/div[2]/div/div/div[3]/div[2]/div[6]/div/div/article[{}]/div[2]/h4/a".format(j))
            element.click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            if "Earnings" in soup.find('h1').getText():  
                title = soup.find('h1').getText()
                transcript_soup = soup.find_all('p')[3:]            
                try:
                    date = soup.find("span", id="date").getText()
                except:
                    date = soup.find('h2').getText()
                transcript = [i.getText() for i in transcript_soup]
                c.execute('INSERT INTO TRANSCRIPTS VALUES (?,?,?,?)',[str(transcript),str(i),str(date),str(title)])
                conn.commit()
                driver.close()
            else:
                pass
    except:
        driver.close()
        pass

In [30]:
#Standardize data for each transcript
df = pd.read_sql_query('SELECT * FROM TRANSCRIPTS', conn)

for i in range(df.shape[0]):
    df['DATE'][i] = df['DATE'][i].replace(".","")
    df['DATE'][i] = df['DATE'][i].replace(",","")


for i in range(df.shape[0]):
    if len(df['DATE'][i]) <20:
        try:
            df['DATE'][i] =  datetime.strptime(df['DATE'][i].strip(), '%b %d %Y')
        except:
            df['DATE'][i] =  datetime.strptime(df['DATE'][i].strip(), '%B %d %Y')
    else:
        try:
            df['DATE'][i] = df['DATE'][i].split("ending",1)[1].strip()
            df['DATE'][i] =  datetime.strptime(df['DATE'][i].strip(), '%b %d %Y')
        except:
            try:
                df['DATE'][i] =  datetime.strptime(df['DATE'][i].strip(), '%B %d %Y')
            except:
                pass

In [33]:
#View Transcripts
df.head()

In [ ]:
#Close Connection
c.execute('DROP TABLE TRANSCRIPTS')
conn.commit()